<center><br> <img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 300px;"></center><br>

<img src="https://logos-world.net/wp-content/uploads/2021/02/Expedia-Emblem.png" style="width: 300px;margin-top: 30px;" align="right">
   <div><center><h1>Projeto: Expedia Hotel Recommendations</h1></center></div>
   <div><center><h4>Processamento de Big Data  -   2º Semestre | 2022/23</h4></center></div>
   <div><center><b>Docentes:</b> João Oliveira <b>|</b> Adriano Lopes</center>
 <br>
    <div><center><h4>Grupo 17</h4></center>
    <div><center> André Silvestre Nº104532 <b>|</b> Rita Matos Nº104936 <b>CDB1</b></center>
 <br>    
 <br>
 <br>
 <br>

<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #075924, #067572, #0371B1, #001D45); 
            padding: 15px; color: white; border-radius: 30000px; text-align: center;">
    <center><h1 style="color: white;font-size: 30px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>4 | Modelos de ML</b></h1></center>
</div>

### PySpark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Expedia Project') \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

Intitializing PySpark ...

23/04/07 13:43:11 WARN Utils: Your hostname, PCAndre resolves to a loopback address: 127.0.1.1; using 172.18.60.45 instead (on interface eth0)
23/04/07 13:43:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/07 13:43:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/07 13:43:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/07 13:43:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark Web UI available at http://172.18.60.45:4042
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1680871393464)
SparkSession available as 'spark'


23/04/07 13:43:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# 📚 Import das Bibliotecas

In [2]:
# Imports Gerais
import os
import sys

# Imports Python
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Imports PySpark
import pyspark.sql.functions as F
from pyspark.sql.types import *

# 🧮 Importar Bases de Dados (parquet)

In [3]:
# Leitura da base de dados "df-expedia-3" guardada anteriormente
df_expedia = spark.read.parquet('df-expedia-3')
# df_expedia = spark.read.parquet('s3://pdb-grupo17-bucket-as/df-expedia-3')

In [4]:
# Leitura da base de dados "small-df-expedia" guardada anteriormente
small_df_expedia = spark.read.parquet('small-df-expedia')
# small_df_expedia = spark.read.parquet('s3://pdb-grupo17-bucket-as/small-df-expedia')

In [5]:
# Confirmação do nº de observações e tipos de dados
print("Nº de Observações:", df_expedia.count())
print("Nº de Observações:", small_df_expedia.count())

df_expedia.printSchema()
small_df_expedia.printSchema()

Nº de Observações: 37455577
Nº de Observações: 11236477
root
 |-- srch_destination_id: integer (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (nullable = true)
 |-- hotel_market: integer (nullable = true)
 |-- pcaFeature_1: double (nullable = true)
 |-- pcaFeature_2: double (nullable = true)
 |-- pcaFeature_3: double (nullable = true)
 |-- tempo_estadia: integer (nullable = true)
 |-- hotel_cluster: integer (nullable = true)

root
 |-- srch_destination_id: integer (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable =

---

# ⚖️ Divisão em Conjunto Treino/Teste

In [8]:
# Divisão em Conjunto Treino/Teste
df_train, df_test = small_df_expedia.randomSplit([0.9, 0.1], 42) # Computador Local
df_train_aws, df_test_aws = df_expedia.randomSplit([0.9, 0.1], 42)     # AWS

n_df_train = df_train.count()             # Conjunto Treino do Computador Local (90%)
n_df_test = df_test.count()               # Conjunto Teste do Computador Local  (10%)

n_df_train_aws = df_train_aws.count()   # Conjunto Treino do AWS   (90%)
n_df_test_aws = df_test_aws.count()     # Conjunto Teste do AWS    (10%)

# Nº de observações em cada parte
print(f"Existem {n_df_train} observações no Conjunto de Treino e {n_df_test} no Conjunto de Teste.")
print(f"Existem {n_df_train_aws} observações no Conjunto de Treino e {n_df_test_aws} no Conjunto de Teste. (AWS)")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Existem 10110707 observações no Conjunto de Treino e 1125432 no Conjunto de Teste.
Existem 33708161 observações no Conjunto de Treino e 3747416 no Conjunto de Teste. (AWS)

---

# 💡 Modelos do Spark ML

- **Random Forest** - Algoritmo de ML que usa várias árvores de decisão para classificar dados.
- **Decision Forest** (Árvore de Decisão) - Algoritmo de ML que usa uma estrutura de árvore para fazer previsões a partir de dados.

In [7]:
# Lista das features a usar
features_list = ['srch_destination_id', 'site_name', 'posa_continent','user_location_country', 
                 'is_package', 'srch_adults_cnt','srch_children_cnt', 'srch_destination_type_id', 
                 'is_booking','hotel_continent', 'hotel_country', 'hotel_market',
                 'pcaFeature_1', 'pcaFeature_2', 'pcaFeature_3','tempo_estadia']

## Random Forest [[1]](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html) [[3]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

<div class="alert alert-block alert-warning">
Amostra do Dataset | <b>Computador</b></div>

- Com o `small_df_expedia` (no **Computador Local**)

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol = 'features')

# Criar um classificador RandomForest
rndfr_sp = RandomForestClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador OneVsRest
pipeline = Pipeline(stages=[assembler, rndfr_sp])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train)
df_pred1_cl = model.transform(df_test)                              # cl - Computador Locar

23/04/06 20:46:30 WARN MemoryStore: Not enough space to cache rdd_54_15 in memory! (computed 8.5 MiB so far)
23/04/06 20:46:30 WARN BlockManager: Persisting block rdd_54_15 to disk instead.
23/04/06 20:46:32 WARN MemoryStore: Not enough space to cache rdd_54_13 in memory! (computed 3.6 MiB so far)
23/04/06 20:46:32 WARN MemoryStore: Not enough space to cache rdd_54_14 in memory! (computed 3.6 MiB so far)
23/04/06 20:46:32 WARN MemoryStore: Not enough space to cache rdd_54_4 in memory! (computed 8.5 MiB so far)
23/04/06 20:46:32 WARN BlockManager: Persisting block rdd_54_14 to disk instead.
23/04/06 20:46:32 WARN BlockManager: Persisting block rdd_54_13 to disk instead.
23/04/06 20:46:32 WARN BlockManager: Persisting block rdd_54_4 to disk instead.
23/04/06 20:46:32 WARN MemoryStore: Not enough space to cache rdd_54_0 in memory! (computed 8.5 MiB so far)
23/04/06 20:46:32 WARN BlockManager: Persisting block rdd_54_0 to disk instead.
23/04/06 20:46:32 WARN MemoryStore: Not enough space t

In [9]:
# Visualização dos Resultados
df_pred1_cl.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

23/04/06 20:47:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/04/06 20:47:08 WARN DAGScheduler: Broadcasting large task binary with size 1005.1 KiB
+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,14.0,1.0,214...|[0.07917509549824...|      64.0|           58|
|[4.0,14.0,1.0,214...|[0.07917509549824...|      64.0|           67|
|[8.0,2.0,3.0,66.0...|[0.21679602593323...|      91.0|           77|
|[8.0,2.0,3.0,66.0...|[0.28042538307994...|      91.0|            7|
|[9.0,17.0,1.0,133...|[0.01357538439369...|      82.0|           20|
|[16.0,2.0,3.0,66....|[0.24578025082366...|      91.0|           47|
|[19.0,2.0,3.0,66....|[0.01558411013500...|      82.0|           64|
|[19.0,2.0,3.0,66....|[0.01558411013500..

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a accuracy, precisão, recall e f1-score
accuracy = evaluator.evaluate(df_pred1_cl)
precision = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

23/04/06 20:47:10 WARN DAGScheduler: Broadcasting large task binary with size 1014.6 KiB
23/04/06 20:47:23 WARN DAGScheduler: Broadcasting large task binary with size 1014.6 KiB
23/04/06 20:47:36 WARN DAGScheduler: Broadcasting large task binary with size 1014.6 KiB
23/04/06 20:47:48 WARN DAGScheduler: Broadcasting large task binary with size 1014.6 KiB
Accuracy =  0.0976365897830051
Precision =  0.05974145978457034
Recall =  0.09763658978300513
F1-Score =  0.03901576881181476


---

<div class="alert alert-block alert-danger">
Dataset Completo | <b>AWS</b></div>

- Com o `df_expedia` (no **AWS**)

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol = 'features')

# Criar um classificador RandomForest
rndfr_sp = RandomForestClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador OneVsRest
pipeline = Pipeline(stages=[assembler, rndfr_sp])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train_aws)
df_pred1_aws = model.transform(df_test_aws)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Visualização dos Resultados
df_pred1_aws.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,2.0,3.0,66.0...|[0.05182137977682...|      64.0|           38|
|[4.0,2.0,3.0,66.0...|[0.05182137977682...|      64.0|           78|
|[4.0,23.0,1.0,1.0...|[0.05202319984521...|      64.0|           30|
|[4.0,30.0,4.0,195...|[0.05074250869623...|      64.0|           25|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|           94|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|           43|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|           16|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[0.24593577865111...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[0.245935778

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a accuracy, precisão, recall e f1-score
accuracy = evaluator.evaluate(df_pred1_aws)
precision = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.101351
Precisão = 0.0573813
Recall = 0.101351
F1-Score = 0.0421619

----

## Decision Forest [[2]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html) [[3]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

<div class="alert alert-block alert-warning">
Amostra do Dataset | <b>Computador</b></div>

- Com o `small_df_expedia` (no **Computador Local**)

In [11]:
from pyspark.ml.classification import DecisionTreeClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train)
df_pred2_cl = model.transform(df_test)

23/04/06 20:48:18 WARN MemoryStore: Not enough space to cache rdd_146_15 in memory! (computed 11.7 MiB so far)
23/04/06 20:48:18 WARN BlockManager: Persisting block rdd_146_15 to disk instead.
23/04/06 20:48:18 WARN MemoryStore: Not enough space to cache rdd_146_15 in memory! (computed 8.9 MiB so far)
23/04/06 20:48:20 WARN MemoryStore: Not enough space to cache rdd_146_12 in memory! (computed 7.8 MiB so far)
23/04/06 20:48:20 WARN BlockManager: Persisting block rdd_146_12 to disk instead.
23/04/06 20:48:20 WARN MemoryStore: Not enough space to cache rdd_146_14 in memory! (computed 5.2 MiB so far)
23/04/06 20:48:20 WARN BlockManager: Persisting block rdd_146_14 to disk instead.
23/04/06 20:48:20 WARN MemoryStore: Not enough space to cache rdd_146_8 in memory! (computed 7.8 MiB so far)
23/04/06 20:48:20 WARN MemoryStore: Not enough space to cache rdd_146_10 in memory! (computed 7.8 MiB so far)
23/04/06 20:48:20 WARN MemoryStore: Not enough space to cache rdd_146_6 in memory! (computed 7

In [12]:
# Visualização dos Resultados
df_pred2_cl.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,14.0,1.0,214...|[0.0,0.0,10048.0,...|      82.0|           58|
|[4.0,14.0,1.0,214...|[0.0,0.0,10048.0,...|      82.0|           67|
|[8.0,2.0,3.0,66.0...|[69638.0,577.0,34...|      91.0|           77|
|[8.0,2.0,3.0,66.0...|[69638.0,577.0,34...|      91.0|            7|
|[9.0,17.0,1.0,133...|[0.0,0.0,10048.0,...|      82.0|           20|
|[16.0,2.0,3.0,66....|[69638.0,577.0,34...|      91.0|           47|
|[19.0,2.0,3.0,66....|[0.0,0.0,10048.0,...|      82.0|           64|
|[19.0,2.0,3.0,66....|[0.0,0.0,10048.0,...|      82.0|           64|
|[19.0,2.0,3.0,66....|[0.0,0.0,10048.0,...|      82.0|           53|
|[19.0,23.0,1.0,1....|[0.0,0.0,10048.0,...|      82.0|           75|
|[19.0,37.0,1.0,0....|[0.0,0.0,10048.0,...|      82.0|           64|
|[21.0,2.0,3.0,62....|[16386.0,0.0

In [13]:
# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a Accuracy, Precision, Recall e F1-Score
accuracy = evaluator.evaluate(df_pred2_cl)
precision = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

Accuracy =  0.09258846007149887
Precision =  0.03283159520980287
Recall =  0.09258846007149886
F1-Score =  0.0320767333617374


---

<div class="alert alert-block alert-danger">
Dataset Completo | <b>AWS</b></div>

- Com o `df_expedia` (no **AWS**)

In [39]:
from pyspark.ml.classification import DecisionTreeClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train_aws)
df_pred2_aws = model.transform(df_test_aws)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Visualização dos Resultados
df_pred2_aws.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,2.0,3.0,66.0...|[526.0,0.0,55200....|      82.0|           38|
|[4.0,2.0,3.0,66.0...|[526.0,0.0,55200....|      82.0|           78|
|[4.0,23.0,1.0,1.0...|[526.0,0.0,55200....|      82.0|           30|
|[4.0,30.0,4.0,195...|[526.0,0.0,55200....|      82.0|           25|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|           94|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|           43|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|           16|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0,2409...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[12225.0,0.0

In [41]:
# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a Accuracy, Precision, Recall e F1-Score
accuracy = evaluator.evaluate(df_pred2_aws)
precision = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.0958517
Precisão = 0.0319509
Recall = 0.0958517
F1-Score = 0.0360443

---

### Conclusão dos Modelos

> Tanto com o *dataset* completo como com parte deste ($30\%$), os resultados dos modelos foram semelhantes, não tendo obtido um desempenho satisfatório para a tarefa de classificação do target `hotel_cluster`.

---

# 🛠️ Tunning do Modelo (Apenas no AWS)

> Como aconselhado pelo docente, efetuamos ***tunning*** ao modelo com melhor performance (de entre os dois modelos testados) de forma a tentar melhorá-lo com diferentes parametrizações.

Assim, fomos procurar formas de o fazer e codificar. [[4]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html) [[5]](https://gist.github.com/colbyford/7758088502211daa90dbc1b51c408762)

In [44]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Definir a grade de parâmetros para testar   | 
#      maxDepth -> define o nº máximo de níveis que a árvore pode ter                  
#      minInstancesPerNode -> define o nº mínimo de amostras necessárias para dividir um nó interno da árvore em dois filhos.
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [2, 5, 10]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()

# Usar CrossValidator para ajustar o modelo com diferentes combinações de parâmetros
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol='hotel_cluster'),
                          numFolds=5)

# Ajustar o modelo com os dados de treino
cvModel = crossval.fit(df_train_aws)

# Prever as classes do conjunto de teste usando o modelo ajustado
df_pred2_tunning = cvModel.transform(df_test_aws)

# Avaliar a precisão do modelo usando a métrica 'accuracy'
evaluator = MulticlassClassificationEvaluator(labelCol='hotel_cluster', metricName='accuracy')
accuracy = evaluator.evaluate(df_pred2_tunning)
precision = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.132363
Precisão = 0.181692
Recall = 0.132363
F1-Score = 0.0846135

---

# 🔗 Referencias Bibliográficas

**[1]** Apache Spark. (n.d.-g). RandomForestClassifier — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html?highlight=randomforestclassifier

**[2]** Apache Spark. (n.d.-a). DecisionTreeClassifier — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html

**[3]** Apache Spark. (n.d.-b). MulticlassClassificationEvaluator — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html

**[4]** Apache Spark. (n.d.-a). ParamGridBuilder — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html

**[5]** Ford, Colby T. (n.d.). SparkML Decision Tree Classification Script with Cross-Validation and Parameter Sweep. Gist. https://gist.github.com/colbyford/7758088502211daa90dbc1b51c408762
